# Assignment #4 - automation
## Advanced Databases
### Kristjan Lõhmus, Rimmo Rõõm


### Preliminaries

Install postgres connector and establish connection

In [1]:
import psycopg2
import pandas.io.sql as sqlio
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
conn = psycopg2.connect(
    host= 'localhost',
    password = "postgres",
    user = "postgres",
    port = 5432,
    )
conn.autocommit = True
cursor = conn.cursor()
cursor.execute('CREATE SCHEMA sales_info;')
cursor.execute('set search_path = "sales_info";')

### A. Constraints programming

#### 1. Implement the structure of this system by creating the tables.

In [3]:
cursor.execute("""
CREATE TABLE Invoice (
                         IdInv SERIAL PRIMARY KEY,
                         DateInv DATE NOT NULL,
                         DatePay DATE,
                         ModePay VARCHAR(50) DEFAULT 'Card' CHECK (ModePay IN ('Check', 'Card', 'Cash')),
                         Status VARCHAR(50) DEFAULT 'Not Paid' CHECK (Status IN ('Paid', 'Not Paid'))
);

-- Create Product table
CREATE TABLE Product (
                         BareCode VARCHAR(50) PRIMARY KEY,
                         Label VARCHAR(100) NOT NULL,
                         Category VARCHAR(50),
                         Stock INT CHECK (Stock > 0)
);

-- Create Customer table
CREATE TABLE Customer (
                          IdCust SERIAL PRIMARY KEY,
                          Name VARCHAR(100) NOT NULL,
                          Phone VARCHAR(20) NOT NULL CHECK (Phone LIKE '00%')
);

-- Create OrderC table
CREATE TABLE OrderC (
                        IdOrd SERIAL PRIMARY KEY,
                        DateOrd DATE NOT NULL,
                        StatusOrd VARCHAR(50) DEFAULT 'Pending' CHECK (StatusOrd IN ('validate', 'Pending', 'Partial')),
                        IdCust INT REFERENCES Customer(IdCust),
                        IdInv INT REFERENCES Invoice(IdInv)
);

-- Create Delivery table
CREATE TABLE Delivery (
                          IdDel SERIAL PRIMARY KEY,
                          DateExp DATE,
                          DateDel DATE
);

-- Create OrderLigne table
CREATE TABLE OrderLigne (
                            IdOrLi SERIAL PRIMARY KEY,
                            Quantity INT NOT NULL CHECK (Quantity > 0),
                            StatusOrLig VARCHAR(50) DEFAULT 'Preparation' CHECK (StatusOrLig IN ('Preparation', 'prepared', 'Supply')),
                            IdOrd INT REFERENCES OrderC(IdOrd),
                            IdDel INT REFERENCES Delivery(IdDel) NULL,
                            BareCode VARCHAR(50) REFERENCES Product(BareCode)
);
""")

Resulting schema:

![Implemented schema](screenshots/01.sales_info.png)

#### 2. Data insertion

##### a. Let’s assume we have customer (‘Alice’,’002736014251’); and four Products (AAA001, ‘Wood Chair’, ‘House’, 10); (AAA002, ‘Iron bed’, ‘House’, 5); (AAA003, ‘Mathematics Book’, ‘Education’, 2); (AAA004, ‘Couch’, ‘House’, 2) in our system.

In [4]:
cursor.execute("""INSERT INTO customer(Name, Phone) values ('Alice','002736014251');""")
sqlio.read_sql_query("SELECT * FROM customer", conn).head()

,idcust,name,phone
0,1,Alice,002736014251


In [5]:
cursor.execute("""
insert into product(BareCode, Label, Category, Stock) values
('AAA001', 'Wood Chair', 'House', 10),
('AAA002', 'Iron bed', 'House', 5),
('AAA003', 'Mathematics Book', 'Education', 2),
('AAA004', 'Couch', 'House', 2);
""")
initial_stock = sqlio.read_sql_query("SELECT * FROM product", conn)
initial_stock.head()

,barecode,label,category,stock
0,AAA001,Wood Chair,House,10
1,AAA002,Iron bed,House,5
2,AAA003,Mathematics Book,Education,2
3,AAA004,Couch,House,2


##### b. Customer Alice order by phone three products concerning (Wood chair, 5); (Mathematics Book’, 4); (Couch, 1). Her primarily choice was to pay by card but unfortunately the payment was rejected so she said she will make the payment by cash after some days.


In [6]:
cursor.execute("""
-- Insert into Invoice table
INSERT INTO Invoice (DateInv, ModePay, Status) VALUES
    (CURRENT_DATE, 'Card', 'Not Paid');

-- Assuming IdCust = 1 and IdInv = 1
-- Insert into OrderC table
INSERT INTO OrderC (DateOrd, StatusOrd, IdCust, IdInv) VALUES
    (CURRENT_DATE, 'Pending', 1, 1);

-- Assuming IdOrd = 1
-- Insert into OrderLigne table
INSERT INTO OrderLigne (Quantity, StatusOrLig, IdOrd, BareCode) VALUES
                                                                    (5, 'Preparation', 1, 'AAA001'),  -- Wood Chair
                                                                    (4, 'Preparation', 1, 'AAA003'),  -- Mathematics Book
                                                                    (1, 'Preparation', 1, 'AAA004');  -- Couch

-- Update the Invoice for payment by cash
UPDATE Invoice
SET ModePay = 'Cash'
WHERE IdInv = 1;
"""
)

In [7]:
sqlio.read_sql_query("""
SELECT 
    c.Name AS CustomerName,
    c.Phone AS CustomerPhone,
    o.DateOrd AS OrderDate,
    o.StatusOrd AS OrderStatus,
    i.DateInv AS InvoiceDate,
    i.DatePay AS PaymentDate,
    i.ModePay AS PaymentMode,
    i.Status AS PaymentStatus,
    ol.Quantity AS ProductQuantity,
    ol.StatusOrLig AS OrderLineStatus,
    p.BareCode AS ProductCode,
    p.Label AS ProductLabel,
    p.Category AS ProductCategory
FROM 
    Customer c
JOIN 
    OrderC o ON c.IdCust = o.IdCust
JOIN 
    Invoice i ON o.IdInv = i.IdInv
JOIN 
    OrderLigne ol ON o.IdOrd = ol.IdOrd
JOIN 
    Product p ON ol.BareCode = p.BareCode
WHERE 
    c.Name = 'Alice' 
    AND c.Phone = '002736014251';

""", conn).head()

,customername,customerphone,orderdate,orderstatus,invoicedate,paymentdate,paymentmode,paymentstatus,productquantity,orderlinestatus,productcode,productlabel,productcategory
0,Alice,002736014251,2024-05-15,Pending,2024-05-15,None,Cash,Not Paid,5,Preparation,AAA001,Wood Chair,House
1,Alice,002736014251,2024-05-15,Pending,2024-05-15,None,Cash,Not Paid,4,Preparation,AAA003,Mathematics Book,Education
2,Alice,002736014251,2024-05-15,Pending,2024-05-15,None,Cash,Not Paid,1,Preparation,AAA004,Couch,House


### B. Automation

#### 1. Implement date field auto-insertion for:


In [8]:
# DateInv
cursor.execute("""
ALTER TABLE Invoice
ALTER COLUMN DateInv SET DEFAULT CURRENT_DATE;
""")

In [9]:
# OrderC
cursor.execute("""
ALTER TABLE OrderC
ALTER COLUMN DateOrd SET DEFAULT CURRENT_DATE;
""")

In [10]:
# DateExp 
cursor.execute("""
ALTER TABLE Delivery
ALTER COLUMN DateExp SET DEFAULT CURRENT_DATE;
""")

In [11]:
# DatePay 
cursor.execute("""
CREATE OR REPLACE FUNCTION update_datepay()
RETURNS TRIGGER AS $$
BEGIN
    IF NEW.Status = 'Paid' AND OLD.Status <> 'Paid' THEN
        NEW.DatePay = CURRENT_DATE;
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

-- Create the trigger for the Invoice table
CREATE TRIGGER set_datepay
BEFORE UPDATE ON Invoice
FOR EACH ROW
EXECUTE FUNCTION update_datepay();
""")

#### 2. Analyze the automation that start when the Invoice status passes to Paid and implement them using triggers. Remember that you have the ability to control the timing of trigger firing. Below is one possible automation schema:

![Task b_02](screenshots/b_02.png)

In [12]:
# First let's create one for the scenario where an invoice gets paid
cursor.execute("""
CREATE OR REPLACE FUNCTION update_order_line_status()
RETURNS TRIGGER AS $$
DECLARE
    order_id INT;
    r_order_line RECORD;
    order_partial BOOLEAN := FALSE;
BEGIN
    IF NEW.Status = 'Paid' THEN
        SELECT IdOrd INTO order_id
            FROM OrderC
        WHERE IdInv = NEW.IdInv
            LIMIT 1;
        FOR r_order_line IN
            SELECT * FROM OrderLigne
            WHERE IdOrd = order_id
        LOOP
            -- Check if product stock is sufficient
            IF (SELECT Stock FROM Product WHERE BareCode = r_order_line.BareCode) >= r_order_line.Quantity THEN
                -- Update order line status to 'prepared' and decrement the stock
                UPDATE OrderLigne
                SET StatusOrLig = 'prepared'
                WHERE IdOrLi = r_order_line.IdOrLi;
                
                UPDATE Product
                SET Stock = Stock - r_order_line.Quantity
                WHERE BareCode = r_order_line.BareCode;
                
                -- Populate delivery date
                UPDATE Delivery
                SET DateDel = CURRENT_DATE
                WHERE IdDel = r_order_line.IdDel;
            ELSE
                -- Update order line status to 'Supply' if stock is insufficient
                UPDATE OrderLigne
                SET StatusOrLig = 'Supply'
                WHERE IdOrLi = r_order_line.IdOrLi;
                
                -- Mark the order as partial
                order_partial := TRUE;
            END IF;
        END LOOP;

        -- Update the overall order status
        IF order_partial THEN
            UPDATE OrderC
            SET StatusOrd = 'Partial'
            WHERE IdOrd = order_id;
        ELSE
            UPDATE OrderC
            SET StatusOrd = 'validate'
            WHERE IdOrd = order_id;
        END IF;
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trigger_update_order_line_status
AFTER UPDATE OF Status ON Invoice
FOR EACH ROW
WHEN (NEW.Status = 'Paid')
EXECUTE FUNCTION update_order_line_status();

""")

In [13]:
cursor.execute("""
CREATE OR REPLACE FUNCTION populate_delivery_date()
RETURNS TRIGGER AS $$
BEGIN
    IF NEW.StatusOrLig = 'prepared' THEN
        UPDATE Delivery
        SET DateDel = CURRENT_DATE
        WHERE Delivery.IdDel = NEW.IdDel;
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE TRIGGER trigger_populate_delivery_date
AFTER UPDATE OF StatusOrLig ON OrderLigne
FOR EACH ROW
WHEN (NEW.StatusOrLig = 'prepared')
EXECUTE FUNCTION populate_delivery_date();
""")

In [14]:
cursor.execute("""
CREATE OR REPLACE FUNCTION handle_supply_status()
RETURNS TRIGGER AS $$
DECLARE
    r_order_line RECORD;
    all_prepared BOOLEAN;
BEGIN
    FOR r_order_line IN 
        SELECT ol.*, p.Stock 
        FROM OrderLigne ol
        JOIN Product p ON ol.BareCode = p.BareCode
        WHERE ol.StatusOrLig = 'Supply'
    LOOP
        -- Check if the updated stock is sufficient
        IF r_order_line.Stock >= r_order_line.Quantity THEN
            -- Update order line status to 'prepared'
            UPDATE OrderLigne
            SET StatusOrLig = 'prepared'
            WHERE IdOrLi = r_order_line.IdOrLi;

            -- Decrement the stock
            UPDATE Product
            SET Stock = Stock - r_order_line.Quantity
            WHERE BareCode = r_order_line.BareCode;

            -- Populate delivery date
            UPDATE Delivery
            SET DateDel = CURRENT_DATE
            WHERE IdDel = r_order_line.IdDel;
            
            all_prepared := NOT EXISTS (
                SELECT 1
                FROM OrderLigne
                WHERE IdOrd = r_order_line.IdOrd AND StatusOrLig != 'prepared'
            );

            IF all_prepared THEN
                UPDATE OrderC
                SET StatusOrd = 'validate'
                WHERE IdOrd = r_order_line.IdOrd;
            ELSE
                UPDATE OrderC
                SET StatusOrd = 'Partial'
                WHERE IdOrd = r_order_line.IdOrd;
            END IF;
        END IF;
    END LOOP;
RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE OR REPLACE  TRIGGER trigger_check_supply_status
AFTER UPDATE OF Stock ON Product
FOR EACH ROW
EXECUTE FUNCTION handle_supply_status();

""")

#### 3. Let’s assume now that Alice is ready for cash payment. Write an update query to set Invoice status and the mode of payment. 

In [15]:
cursor.execute("""
UPDATE Invoice
SET ModePay = 'Cash', Status = 'Paid'
WHERE IdInv IN (
    SELECT o.IdInv
    FROM OrderC o
    JOIN Customer c ON o.IdCust = c.IdCust
    WHERE c.Name = 'Alice' AND c.Phone = '002736014251'
);
""")

#### 4. If everything performs exactly, then:

In [16]:
# OrderStatus to partial

sqlio.read_sql_query("""
SELECT idord, statusord FROM OrderC
""",conn).head()

,idord,statusord
0,1,Partial


In [17]:
# Order ligne status for Wood chair to prepared

sqlio.read_sql_query("""
SELECT statusorlig FROM OrderLigne ol left join Product p on ol.bareCode = p.BareCode
WHERE p.label = 'Wood Chair'
""",conn).head()

,statusorlig
0,prepared


In [18]:
# Order ligne status for Couch to prepared

sqlio.read_sql_query("""
SELECT statusorlig FROM OrderLigne ol left join Product p on ol.bareCode = p.BareCode
WHERE p.label = 'Couch'
""",conn).head()

,statusorlig
0,prepared


In [19]:
# Order ligne status for Mathematics Book to Supply

sqlio.read_sql_query("""
SELECT statusorlig FROM OrderLigne ol left join Product p on ol.bareCode = p.BareCode
WHERE p.label = 'Mathematics Book'
""",conn).head()

,statusorlig
0,Supply


In [20]:
# A tuple in the delivery whose ID is the same as that used in the Id delivery in order ligne  for the ‘wood chair’ and ‘Couh’.
# The Id delivery for product ‘Mahematic book’ is still tonull.

sqlio.read_sql_query("""
SELECT 
    p.Label AS ProductLabel,
    ol.IdDel AS DeliveryId
FROM 
    OrderLigne ol
JOIN 
    Product p ON ol.BareCode = p.BareCode
LEFT JOIN 
    Delivery d ON ol.IdDel = d.IdDel
WHERE 
    (p.Label IN ('Wood Chair', 'Couch') AND ol.IdDel IS NOT NULL)
    OR (p.Label = 'Mathematics Book' AND ol.IdDel IS NULL);

""",conn).head()

,productlabel,deliveryid
0,Mathematics Book,None


In [21]:
# Stocks of these products also should decrease

new_stock = sqlio.read_sql_query("""
SELECT barecode, label, stock as new_stock from Product
""",conn)
initial_stock.rename(columns={'stock': 'old_stock'}, inplace=True)
pd.merge(initial_stock[['barecode', 'old_stock']], new_stock, on='barecode', how='outer')[['barecode', 'label', 'old_stock','new_stock']]

,barecode,label,old_stock,new_stock
0,AAA001,Wood Chair,10,5
1,AAA002,Iron bed,5,5
2,AAA003,Mathematics Book,2,2
3,AAA004,Couch,2,1


#### 5.  Write a query to provision the stock of Mathematic book. Let’s say that we add to the stock 10.

In [22]:
cursor.execute("""
UPDATE Product
SET Stock = Stock + 10
WHERE Label = 'Mathematics Book';
""")

In [23]:
# Check your database content mainly if you have OrderStatus to Validate
sqlio.read_sql_query("""
SELECT idord, statusord FROM OrderC
""",conn).head()

,idord,statusord
0,1,validate
